In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import markers

In [ ]:
sns.set_style('whitegrid')
all_shapes = list(markers.MarkerStyle.markers.keys())

In [ ]:
raw_data = pd.read_csv('../data/overall_homelessness.csv', header=None)
list_states = ['NY', 'CA', 'MA', 'WA', 'PA', 'OH', 'AZ', 'TX', 'OR', 'IL', 'MI', 'CO', 'GA', 'FL']
